# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [181]:
# import libraries
import pandas as pd
import numpy as np
import re
import sqlite3
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [203]:
verb = StartingVerbExtractor()
verb.transform(test)

,0
0,False


In [200]:
test = X_test[0]

In [190]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [196]:
stop_words = set(stopwords.words('english'))

def load_data():
    engine = create_engine('sqlite:///qaissSQL.db')
    df = pd.read_sql('SELECT * FROM qaiss_df_ready', engine)
    X = df.message
    y = df.iloc[:, 4:]
    return X, y

def tokenize(text):
    #loads one text at a  time, need to loop over.
    text = word_tokenize(text)
    lem = WordNetLemmatizer()
    
    token_list = []
    for t in text:
        text = lem.lemmatize(t).lower()
        if text not in stop_words:
            token_list.append(text)
    
    return token_list

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred)
    accuracy = (y_pred == y_test).mean().mean()
    

    print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    


def model_pipline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline
    
    
def model_pipline_2():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_test_2)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline


def model_pipline_2_start_verb():
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize_test_2)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline


def tokenize_test_2(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    char = '[^A-Za-z0-9]+'
    detected_urls = re.findall(url_regex, text)
    chars_found = re.findall(char, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    for char in chars_found:
        text = text.replace(char, ' ')
    
    text = word_tokenize(text)
    lem = WordNetLemmatizer()
    
    token_list = []
    for t in text:
        text = lem.lemmatize(t).lower()
        if text not in stop_words:
            token_list.append(text)
    
    return token_list
            
    
def main():
    X,y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    pipeline = model_pipline()
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    #return y_pred

    display_results(y_test, y_pred)
    
    
def main_2():
    X,y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    pipeline = new_model_pipeline()
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    #return y_pred

    display_results(y_test, y_pred)

    
def new_model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

In [183]:
main_2()

Labels: [0 1 2]
Accuracy: 0.944580069847


In [197]:
main_2()

IndexError: list index out of range

In [99]:
type(tests)

str

In [ ]:
    X, Y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, Y)

    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    
    clf = RandomForestClassifier()

    # train classifier
    X_train_counts = vect.fit_transform(X_train)
    X_train_tfidf = tfidf.fit_transform(X_train_counts)
    clf.fit(X_train_tfidf, y_train)

    # predict on test data
    X_test_counts = vect.transform(X_test)
    X_test_tfidf = tfidf.transform(X_test_counts)
    y_pred = clf.predict(X_test_tfidf)


In [132]:
#Y.head()

#pd.get_dummies(Y)

In [92]:
MultiOutputClassifier?

In [97]:
m = MultiLabelBinarizer().fit_transform(Y)

In [131]:
Y.values.shape

(26216, 36)

In [123]:
multi = MultiOutputClassifier()

TypeError: __init__() missing 1 required positional argument: 'estimator'

In [50]:
testing

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

In [ ]:
filtered = [fow w in token_list]

In [61]:
token_list

NameError: name 'token_list' is not defined

In [112]:
xx.shape

(100, 20)

In [114]:
yy.shape

(100, 3)

### 2. Write a tokenization function to process your text data

In [64]:

def tokenize(text):
    text = word_tokenize(text)
    lem = WordNetLemmatizer()
    
    token_list = []
    for t in text:
        text = lem.lemmatize(t).lower()
        if text not in stop_words:
            token_list.append(text)
    
    return token_list

In [68]:
testing = X[0:5]
for t in testing:
    ans = tokenize(t)
    
ans

['say',
 ':',
 'west',
 'side',
 'haiti',
 ',',
 'rest',
 'country',
 'today',
 'tonight']

In [54]:
testing = X[0:1]
for t in testing:
    ans = tokenize(t)
    
ans

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [20]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [30]:
len(Y.iloc[0,:])

36

In [34]:
Y.iloc[0,:]

related                   1
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       